In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.regularizers import l2
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.utils.training_utils import multi_gpu_model
from keras.callbacks import LearningRateScheduler
from keras import optimizers
from keras import backend as K
import tensorflow as tf

In [ ]:
train_data_dir = 'C:\\Users\\Parth\\Desktop\\dataset\\male\\train'
validation_data_dir = 'C:\\Users\\Parth\\Desktop\\dataset\\male\\valid'
epochs = 250
batch_size = 128
img_size = 256
num_gpus = 2
initial_learning_rate = 5e-4
weight_decay = 0.0005

In [ ]:
def poly_decay(epoch):
    # initialize the maximum number of epochs, base learning rate,
    # and power of the polynomial
    maxEpochs = epochs
    baseLR = initial_learning_rate
    power = 1.0 # linear decay
 
    # compute the new learning rate based on polynomial decay
    alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power
 
    # return the new learning rate
    return alpha

callbacks = [LearningRateScheduler(poly_decay)]

In [ ]:
# we'll store a copy of the model on *every* GPU and then combine
# the results from the gradient updates on the CPU
with tf.device("/cpu:0"):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
    print('Model loaded.')
    
    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    # and a logistic layer -- let's say we have 2 classes
    predictions = Dense(2, activation='softmax')(x)
    
    # initialize the model
    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)
    
model = multi_gpu_model(model, gpus=num_gpus)

In [ ]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# compile the model (should be done *after* setting layers to non-trainable)
opt = optimizers.RMSprop(lr=initial_learning_rate)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# prepare data augmentation configuration
train_datagen = image.ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = image.ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
# train the model
model.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator, callbacks=callbacks, verbose = 1)

In [ ]:
model.save_weights('inception_finetuned_male_dropout_l2.h5')